In [62]:
from ptsa.data.TimeSeriesX import TimeSeriesX
from ptsa.data.common import xr
import numpy as np
import glob
import scipy

In [135]:
import pandas as pd
rhino_root = '/Volumes/RHINO'
from ptsa.data.readers import BaseEventReader
import tables
events = BaseEventReader(filename=rhino_root + '/data/eeg/scalp/ltp/ltpFR2/behavioral/events/events_all_LTP093.mat',common_root='data').read()
events = pd.DataFrame.from_records([e for e in events],columns = events.dtype.names)

In [136]:
print events

Index([u'subject', u'session', u'trial', u'type', u'serialpos',
       u'begin_distractor', u'begin_math_correct', u'final_distractor',
       u'final_math_correct', u'item', u'itemno', u'recalled', u'intruded',
       u'rectime', u'intrusion', u'mstime', u'msoffset', u'eegfile',
       u'eegoffset', u'artifactMS', u'artifactNum', u'artifactFrac',
       u'artifactMeanMS', u'badEvent', u'badEventChannel'],
      dtype='object')


In [137]:
print events['rectime']

0        -999.0
1        -999.0
2        -999.0
3        -999.0
4        -999.0
5        -999.0
6        -999.0
7        -999.0
8        -999.0
9        -999.0
10       -999.0
11       -999.0
12       -999.0
13       -999.0
14       -999.0
15       -999.0
16       -999.0
17       -999.0
18       -999.0
19       -999.0
20       -999.0
21       -999.0
22       -999.0
23       -999.0
24       -999.0
25       -999.0
26       1014.0
27       1711.0
28       2679.0
29       3811.0
          ...  
19750    -999.0
19751    -999.0
19752    -999.0
19753    -999.0
19754    -999.0
19755    -999.0
19756    -999.0
19757    -999.0
19758    -999.0
19759    -999.0
19760    -999.0
19761    -999.0
19762    -999.0
19763    -999.0
19764    -999.0
19765    -999.0
19766    -999.0
19767    -999.0
19768    -999.0
19769    -999.0
19770    -999.0
19771    -999.0
19772     441.0
19773     982.0
19774    1537.0
19775    2937.0
19776    3582.0
19777    4126.0
19778    5315.0
19779    5806.0
Name: rectime, Length: 1

In [51]:
a = np.arange(10)
b = np.linspace(0,1,10)
dataframe = pd.DataFrame({'a':a, 'b':b})
print dataframe.loc[0]

a    0.0
b    0.0
Name: 0, dtype: float64


In [52]:
# xarray
a = xr.DataArray(np.random.randn(2, 3))
print a

<xarray.DataArray (dim_0: 2, dim_1: 3)>
array([[-1.681881, -1.562026, -0.654929],
       [ 1.446629,  1.98721 ,  1.950876]])
Dimensions without coordinates: dim_0, dim_1


In [63]:
def get_ltpFR2_precall(files_ltpFR2):

    """This function gets the recall probability of each word for each participant
    dimenstions should be N participants * 576 words """

    # determining whether each word is recalled or not when it is presented to the participant
    # dimenstions should be (576 words * times word was presented (23) for each participant

    def count_recalls(rec_mat, pres_mat):
        word_id_recalled_or_not_this_part = []
        # print("rec mat", rec_mat)
        for word_id in np.unique(pres_mat):
            #print("Word ID:", word_id)
            word_id_recalled_or_not_this_part.append(rec_mat[np.where((pres_mat) == word_id)])
        #print("Shape Word id this part", np.shape(word_id_recalled_or_not_this_part))
        #print(word_id_recalled_or_not_this_part)
        return word_id_recalled_or_not_this_part

    all_participants_all_recalled = []

    # Do this for every participant
    for f in files_ltpFR2:
        # Read in data
        test_mat_file = scipy.io.loadmat(f, squeeze_me=True, struct_as_record=False)
        if isinstance(test_mat_file['data'], np.ndarray):
            #print('Skipping...')
            continue
        session_mat = test_mat_file['data'].session
        # print(len(session_mat))
        # print(np.bincount(session_mat))
        if len(session_mat) < 576:
            print('Skipping because participant did not finish...')
            continue
        else:
            print(f)
            pres_mat = test_mat_file['data'].pres_itemnos.astype('int16')
            pres_mat = pres_mat[np.where(session_mat != 24)]
            rec_mat = test_mat_file['data'].pres.recalled
            # rec_mat = rec_mat[np.where(session_mat != 24)]
            # print("Legth of pres_mat", (pres_mat).shape)
            # print("Length of rec_mat", (rec_mat).shape)

        # print("Pres mat", pres_mat)
        # print("Rec mat", rec_mat)
        #print(rec_mat.shape)

        # For each list of this participant's data
        this_participant_probs = []

        word_id_recalled_this_part = count_recalls(rec_mat, pres_mat)

        # Append this partic p_rec to everyone else
        all_participants_all_recalled.append(np.nanmean(word_id_recalled_this_part, axis = 1))

    #Get the average of all participants
    return all_participants_all_recalled

In [90]:
files_ltpFR2 = glob.glob(rhino_root + '/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP*.mat')
print files_ltpFR2[0:10]
# all_participants_all_recalled = get_ltpFR2_precall(files_ltpFR2[0:15])
# print(all_participants_all_recalled)
# print("Shape of all participants all recalled", np.shape(all_participants_all_recalled))

['/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP214.mat', '/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP320.mat', '/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP309.mat', '/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP281.mat', '/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP242.mat', '/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP308.mat', '/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP325.mat', '/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP315.mat', '/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP314.mat', '/Volumes/RHINO/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP274.mat']


In [119]:
path = rhino_root + '/data/eeg/scalp/ltp/ltpFR2/behavioral/data/stat_data_LTP093.mat'
datamat = scipy.io.loadmat(path)
print type(datamat)

<type 'dict'>


In [138]:
path = rhino_root + '/data/eeg/scalp/ltp/ltpFR2/subj_and_sess_info.mat'
datamat = scipy.io.loadmat(path)


In [169]:

print datamat['sess_info'][0,0]['day']


[[  2.]
 [  8.]
 [  7.]
 ..., 
 [ 12.]
 [  3.]
 [ 30.]]
